In [1]:
from data import get_challenge_points, get_challenge_labels, get_synth_points


In [2]:
%cd ..

c:\Users\ksush\attacks\MIDSTModels


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from midst_models.single_table_TabDDPM.complex_pipeline import (
    clava_clustering,
    clava_training,
    clava_load_pretrained,
    clava_synthesizing,
    load_configs,
    clava_attacking
)
from midst_models.single_table_TabDDPM.pipeline_modules import load_multi_table
from midst_models.single_table_TabDDPM.complex_pipeline import tabddpm_whitebox_load_pretrained
import warnings
warnings.filterwarnings('ignore')


In [4]:
from torchmetrics.classification import BinaryAUROC, BinaryROC
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import logging


### Binary classifier

In [5]:
import os
import logging
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def attack(base_dir, inference=False, model_path="trained_model.pkl"):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    logger.info("Loading the attacked model...")

    # Initialize attacker
    phases = ["dev", "final"] if inference else ["train"]
    logger.info("Attack start...")

    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)

            challenge_points = get_challenge_points(path)  # Shape: (8, 200)
            X = challenge_points.reshape(-1, challenge_points.shape[-1])  # Flatten if needed
            
            if not inference:
                challenge_labels = np.array(get_challenge_labels(path))  # Ensure it's a NumPy array
                y = challenge_labels.ravel()  # Ensure labels are 1D
            
            if not inference:
                # Split data for validation
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
                
                # Train models on all data and save
                model = lgb.LGBMClassifier()
                model.fit(X, y)
                
                # Evaluate model
                y_train_pred = model.predict(X_train)
                train_accuracy = accuracy_score(y_train, y_train_pred)
                logger.info(f"Training Accuracy: {train_accuracy:.4f}")
                
                joblib.dump(model, model_path)
                logger.info(f"Model saved to {model_path}")
            else:
                # Load trained model for inference
                if not os.path.exists(model_path):
                    logger.error(f"Model file {model_path} not found!")
                    return
                
                model = joblib.load(model_path)
                y_pred = model.predict(X)
                
                # Save predictions per model folder
                prediction_file = os.path.join(path,"prediction.csv")
                pd.DataFrame(y_pred).to_csv(prediction_file, index=False, header=False)
                logger.info(f"Predictions saved to {prediction_file}")


In [6]:
import os
import logging
import numpy as np
import joblib
from sklearn.preprocessing import QuantileTransformer

def train_and_save_quantile_transformer(base_dir, transformer_path="quantile_transformer.pkl", seed=42):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    logger.info("Collecting challenge data for normalization...")

    X_all = []  # Container for all challenge data
    for phase in ["train", "dev", "final"]:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
            challenge_points = get_challenge_points(path)  # Shape: (8, 200)
            synth_points = get_synth_points(path)  # Shape: (8, 200)

            X_all.append(challenge_points.reshape(-1, challenge_points.shape[-1]))
            X_all.append(synth_points.reshape(-1, synth_points.shape[-1]))

        if not X_all:
            logger.warning("No valid data found for normalization.")
            return
        
    X_all = np.vstack(X_all)
    
    # Initialize and fit the QuantileTransformer
    normalizer = QuantileTransformer(
        output_distribution="normal",
        n_quantiles=max(min(X_all.shape[0] // 30, 1000), 10),
        subsample=int(1e9),
        random_state=seed,
    )
    normalizer.fit(X_all)
    
    # Save the trained transformer
    joblib.dump(normalizer, transformer_path)
    logger.info(f"QuantileTransformer saved to {transformer_path}")


In [7]:
train_and_save_quantile_transformer(base_dir="tabddpm_white_box",)

In [8]:
attack(base_dir="tabddpm_white_box", inference=True)

## Newer PIA Attack

In [9]:
import torch
import numpy as np
import logging
from typing import Dict, Type
import midst_models.attack.components as components
import numpy as np
import re

class EpsGetter(components.EpsGetter):
    def __call__(self, xt: torch.Tensor, condition: torch.Tensor = None, noise_level=None, t: int = None) -> torch.Tensor:
        # Access the diffusion model from the dictionary structure
        model = self.model

        t = torch.ones([xt.shape[0]], device=xt.device).long() * t
        return model(xt, timesteps=t)

ATTACKERS: Dict[str, Type[components.DDIMAttacker]] = {
    "PIA": components.PIA,
    "PIAN": components.PIAN,
}


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def get_FLAGS():
    def FLAGS(x): return x
    FLAGS.T = 50000
    FLAGS.ch = 128
    FLAGS.ch_mult = [1, 2, 2, 2]
    FLAGS.attn = [1]
    FLAGS.num_res_blocks = 2
    FLAGS.dropout = 0.01
    FLAGS.beta_1 = 0.0001
    FLAGS.beta_T = 0.02

    return FLAGS

FLAGS = get_FLAGS()


### GENERATE DATA

In [10]:
%cd ..

c:\Users\ksush\attacks


In [11]:
import os
import json
import glob

def process_tabddpm_folders(base_path, attacker_name="PIA", interval=200, attack_num=1):
    """
    Processes all 'tabddpm_{n}' folders inside the given base path.

    Args:
        base_path (str): The base directory containing 'tabddpm_{n}' folders.
    """
    # Find all folders matching the pattern 'tabddpm_{n}'
    tabddpm_folders = glob.glob(os.path.join(base_path, "tabddpm_*"))

    for path in tabddpm_folders:
        for i in range(0, 500):
            print(f"\nProcessing folder: {path}")

            # Load pretrained model
            model = tabddpm_whitebox_load_pretrained(path)
            # model = model[(None, 'trans')]['diffusion']._denoise_fn

            # Load config
            config_path = os.path.join(path, "trans.json")
            configs, save_dir = load_configs(config_path)

            # Display config
            print("\n===== Configurations =====")
            print(json.dumps(configs, indent=4))

            # Load dataset
            tables, relation_order, dataset_meta = load_multi_table(path, train_data="challenge")

            # Display table keys
            print("\n===== Table Keys =====")
            print(list(tables.keys()))

            # Display clustering parameters
            params_clustering = configs["clustering"]
            print("\n===== Clustering Parameters =====")
            for key, val in params_clustering.items():
                print(f"{key}: {val}")

            # Perform clustering
            tables, all_group_lengths_prob_dicts = clava_clustering(
                tables, relation_order, save_dir, configs
            )

            print(f"Finished processing {path}\n" + "=" * 50)
            attacker = ATTACKERS[attacker_name](
                torch.from_numpy(np.linspace(FLAGS.beta_1, FLAGS.beta_T, FLAGS.T)).to(DEVICE), interval, attack_num, EpsGetter(model), None, None)

            # Launch training from scratch
            # clava_attacking(tables, relation_order, path, configs, attacker, model)

            # # Generate synthetic data from scratch
            cleaned_tables, synthesizing_time_spent, matching_time_spent = clava_synthesizing(
                tables,
                relation_order,
                save_dir,
                all_group_lengths_prob_dicts,
                model,
                configs,
                # sample_scale=1 if "debug" not in configs else configs["debug"]["sample_scale"],
                sample_scale=1.0,
                save_distances=path,
                attacker=attacker
            )
            # Cast int values that saved as string to int for further evaluation
            for key in cleaned_tables.keys():
                for col in cleaned_tables[key].columns:
                    if cleaned_tables[key][col].dtype == "object":
                        try:
                            cleaned_tables[key][col] = cleaned_tables[key][col].astype(int)
                        except ValueError:
                            print(f"Column {col} cannot be converted to int.")
            print(f"Finished synthesizing data for {path}\n" + "=" * 50)
            cleaned_tables.to_csv(os.path.join(path, f"synth_data_{i}.csv"), index=False)
            # print("Finished performing the MIA")


# Define base directories
train_base = r"C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\train"
dev_base = r"C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\dev"
final_base = r"C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\final"

# Process all tabddpm folders in dev and final
process_tabddpm_folders(train_base, interval=5800, attack_num=1, attacker_name="PIA", )
# process_tabddpm_folders(dev_base, interval=5800, attack_num=1, attacker_name="PIA", )
# process_tabddpm_folders(final_base, interval=5800, attack_num=1, attacker_name="PIA", )



Processing folder: C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\train\tabddpm_1
Checkpoint found, loading...

===== Configurations =====
{
    "general": {
        "data_dir": "/projects/aieng/midst_competition/data/tabddpm/tabddpm_1",
        "exp_name": "train_1",
        "workspace_dir": "/projects/aieng/midst_competition/data/tabddpm/tabddpm_1/workspace",
        "sample_prefix": "",
        "test_data_dir": "/projects/aieng/midst_competition/data/tabddpm/tabddpm_1"
    },
    "clustering": {
        "parent_scale": 1.0,
        "num_clusters": 50,
        "clustering_method": "both"
    },
    "diffusion": {
        "d_layers": [
            512,
            1024,
            1024,
            1024,
            1024,
            512
        ],
        "dropout": 0.0,
        "num_timesteps": 2000,
        "model_type": "mlp",
        "iterations": 200000,
        "batch_size": 4096,
        "lr": 0.0006,
        "gaussian_loss_type": "mse",
        "weight_decay": 1e-05,


KeyboardInterrupt: 

In [ ]:
model = tabddpm_whitebox_load_pretrained(r"C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\dev\tabddpm_31")

In [ ]:
model[(None, 'trans')]['dataset']

## Save MIA outputs

In [26]:
def normalize_scores(member_distances, nonmember_distances):
    # If the distances are 2D tensors, take mean along dimension 1
    if len(member_distances.shape) > 1:
        member_distances = member_distances.mean(dim=1)
    if len(nonmember_distances.shape) > 1:
        nonmember_distances = nonmember_distances.mean(dim=1)
    
    max_dist = max(member_distances.max().item(), nonmember_distances.max().item())
    member_probs = 1 - (member_distances / max_dist)
    nonmember_probs = 1 - (nonmember_distances / max_dist)
    return member_probs, nonmember_probs

def process_saved_distances(base_path):
    results = []
    auc_values = []  # List to store AUC values for each folder
    roc_values = []  # List to store ROC values for each folder
    
    tabddpm_folders = glob.glob(os.path.join(base_path, "tabddpm_*"))
    
    for path in tabddpm_folders:
        print(f"\nProcessing folder: {path}")
        
        distances_path = os.path.join(path, "train_saved_outputs_new_distances.pth")
        if not os.path.exists(distances_path):
            print(f"Skipped {path} - distances file not found.")
            continue
        
        distances_0_200 = torch.load(distances_path)
        
        # Check if distances_0_200 is a list with a single tensor
        if isinstance(distances_0_200, list) and len(distances_0_200) == 1:
            distances_0_200 = distances_0_200[0]
        
        # Stack the tensors if they're not already stacked
        if isinstance(distances_0_200, list):
            distances_0_200 = torch.stack(distances_0_200)
        
        label_path = os.path.join(path, "challenge_label.csv")
        if os.path.exists(label_path):
            label_df = pd.read_csv(label_path)
            labels = label_df.iloc[:, 0].values
            members = labels == 1
            non_members = labels == 0
            
            member_scores, nonmember_scores = normalize_scores(distances_0_200[members], distances_0_200[non_members])
            scores = torch.cat([member_scores, nonmember_scores])
            
            # Convert to NumPy for saving
            scores_np = scores.numpy()
            pd.DataFrame(scores_np).to_csv(os.path.join(path, "prediction.csv"), index=False, header=False)
            
            # Store the averaged scores
            results.append(scores)
            
            # Calculate AUROC
            auroc = BinaryAUROC()(scores, torch.cat([torch.ones_like(member_scores), 
                                                    torch.zeros_like(nonmember_scores)]).long()).item()
            auc_values.append(auroc)
            
            roc = BinaryROC()(scores, torch.cat([torch.ones_like(member_scores), 
                                               torch.zeros_like(nonmember_scores)]).long())
            
            fpr, tpr, _ = roc
            fpr_threshold = 0.1
            tpr_at_1_fpr = tpr[(fpr < fpr_threshold).sum() - 1].item()
            
            print(f"Folder: {path}")
            print(f"AUROC: {auroc:.4f}")
            print(f"TPR at 10% FPR: {tpr_at_1_fpr:.4f}")
            roc_values.append(tpr_at_1_fpr)
        else:
            print(f"No labels found for {path}, processing distances without AUROC.")
            scores, _ = normalize_scores(distances_0_200, distances_0_200)
            scores_np = scores.numpy()
            results.append(scores)
            pd.DataFrame(scores_np).to_csv(os.path.join(path, "prediction.csv"), index=False, header=False)
        
        print("=" * 50)
    
    # Calculate the average AUC after all runs
    if auc_values:
        avg_auc = sum(auc_values) / len(auc_values)
        avg_roc = sum(roc_values) / len(roc_values)
        print(f"\nAverage TPR at 10% FPR: {avg_roc:.4f}")
        print(f"\nAverage AUROC: {avg_auc:.4f}")
    else:
        print("\nNo AUC values calculated.")
    
    return results


In [ ]:
# train_results = process_saved_distances(train_base)
dev_results = process_saved_distances(dev_base)
final_results = process_saved_distances(final_base)

In [ ]:
torch.load(os.path.join(train_base, "tabddpm_1", "train_saved_outputs_new_distances.pth"))

In [ ]:

# Distance function
def distance(x0, x1, lp=2):
    return ((x0 - x1).abs()**lp).flatten(1).sum(dim=-1)

# Normalize distances to [0,1] range for probability scores
def normalize_scores(member_distances, nonmember_distances):
    max_dist = max(member_distances.max().item(), nonmember_distances.max().item())
    member_probs = 1 - (member_distances / max_dist)
    nonmember_probs = 1 - (nonmember_distances / max_dist)
    return member_probs, nonmember_probs

def process_saved_outputs(base_path):
    """
    Processes 'saved_outputs_0_200_1000.pth' in each 'tabddpm_{n}' folder inside the given base path,
    calculates AUROC, TPR at 10% FPR, and saves the predictions in 'predictions.csv'.
    
    Args:
        base_path (str): The base directory containing 'tabddpm_{n}' folders.
    """
    # Find all folders matching the pattern 'tabddpm_{n}'
    tabddpm_folders = glob.glob(os.path.join(base_path, "tabddpm_*"))

    for path in tabddpm_folders:
        print(f"\nProcessing folder: {path}")

        # Load saved_outputs from the folder
        saved_outputs_path = os.path.join(path, "saved_outputs_0_200_1000.pth")
        if not os.path.exists(saved_outputs_path):
            print(f"Skipped {path} - saved_outputs file not found.")
            continue
        
        saved_outputs = torch.load(saved_outputs_path)

        # Check for challenge labels
        label_path = os.path.join(path, "challenge_label.csv")
        if os.path.exists(label_path):
            label_df = pd.read_csv(label_path)
            labels = label_df.iloc[:, 0].values  # Convert to numpy array
            members = labels == 1
            non_members = labels == 0

            # Compute distances for T = 0 vs 1000 and T = 0 vs 200
            distances_0_1000 = distance(saved_outputs[0], saved_outputs[1000])

            distances_0_200 = distance(saved_outputs[0], saved_outputs[200])

            # Normalize distances and compute scores
            member_scores, nonmember_scores = normalize_scores(distances_0_200[members], distances_0_200[non_members])

            # Create labels (1 for members, 0 for non-members)
            scores = torch.cat([member_scores, nonmember_scores])
            labels = torch.cat([torch.ones_like(member_scores), torch.zeros_like(nonmember_scores)]).long()

            # Compute predictions
            predictions = (scores).long()

            # Save predictions and scores to CSV
            predictions_df = pd.DataFrame({'score': scores.numpy()})
            predictions_df.to_csv(os.path.join(path, "predictions.csv"), index=False, header=False)

            # Compute AUROC
            auroc = BinaryAUROC()(scores, labels).item()

            # Compute ROC curve
            roc = BinaryROC()(scores, labels)
            fpr, tpr, _ = roc  # Unpack outputs

            # Extract TPR at 10% FPR
            fpr_threshold = 0.1
            tpr_at_1_fpr = tpr[(fpr < fpr_threshold).sum() - 1].item()

            # Print results
            print(f"Folder: {path}")
            print(f"AUROC: {auroc:.4f}")
            print(f"TPR at 10% FPR: {tpr_at_1_fpr:.4f}")
            print(f"Predictions saved to {os.path.join(path, 'predictions.csv')}")
        else:
            # If challenge_label.csv is not found, just save the predictions based on the available outputs
            print(f"No labels found for {path}, saving predictions without AUROC.")
            distances_0_200 = distance(saved_outputs[0], saved_outputs[200])
            # print(f"Distances 0 vs 1000: {distances_0_200.shape} from {saved_outputs[0].shape} and {saved_outputs[200].shape}")

            # Normalize and save predictions
            member_scores, nonmember_scores = normalize_scores(distances_0_200, distances_0_200)  # No label distinction

            # Save predictions to CSV
            scores = torch.cat([member_scores])
            predictions_df = pd.DataFrame({'score': scores.numpy()})
            predictions_df.to_csv(os.path.join(path, "predictions.csv"), index=False, header=False)

            print(f"Predictions [{predictions_df.shape}] saved to {os.path.join(path, 'predictions.csv')}")

        print("=" * 50)


# Define base directories for dev and final
dev_base = r"C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\dev"
final_base = r"C:\Users\ksush\attacks\MIDSTModels\tabddpm_white_box\final"

# Process all saved_outputs in dev and final
process_saved_outputs(dev_base)
process_saved_outputs(final_base)


In [ ]:
import torch
saved_outputs = torch.load("saved_outputs_0_200_1000.pth")

In [ ]:

# Sample distance function
def distance(x0, x1, lp=2):
    return ((x0 - x1).abs()**lp).flatten(1).sum(dim=-1)


# Graph A: Amount of noise predicted per point at different values of T
x_values = sorted(saved_outputs.keys())
y_values = [saved_outputs[t].abs().mean(dim=1).mean().item() for t in x_values]

plt.figure(figsize=(8, 5))
plt.plot(x_values, y_values, marker='o', linestyle='-', label='Avg. Noise per Point')
plt.xlabel("T values")
plt.ylabel("Average Noise")
plt.title("Noise Predicted per Point across Different T Values")
plt.legend()
plt.grid()
plt.show()

# Graph B: Distance between T=0 & T=1000 and T=0 & T=200
distances_0_1999 = distance(saved_outputs[0], saved_outputs[1999])
distances_0_1800= distance(saved_outputs[0], saved_outputs[1800])
distances_0_1600 = distance(saved_outputs[0], saved_outputs[1600])
distances_0_1400 = distance(saved_outputs[0], saved_outputs[1400])
distances_0_1200 = distance(saved_outputs[0], saved_outputs[1200])
distances_0_1000 = distance(saved_outputs[0], saved_outputs[1000])
distances_0_800 = distance(saved_outputs[0], saved_outputs[800])
distances_0_600 = distance(saved_outputs[0], saved_outputs[600])
distances_0_400 = distance(saved_outputs[0], saved_outputs[400])
distances_0_200 = distance(saved_outputs[0], saved_outputs[200])

plt.figure(figsize=(10, 5))
plt.hist(distances_0_1999.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=1999)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()

# TODO: repeat for other values of T

plt.figure(figsize=(10, 5))
plt.hist(distances_0_1800.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=1800)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.hist(distances_0_1600.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=1600)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()

plt.figure(figsize=(10, 5))
plt.hist(distances_0_1400.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=1400)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.hist(distances_0_1000.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=1000)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.hist(distances_0_800.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=800)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.hist(distances_0_600.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=600)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
plt.hist(distances_0_400.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=400)')
plt.hist(distances_0_200.numpy(), bins=50, alpha=0.6, label='Dist(T=0, T=200)')
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Distribution of Distances between Noise Predictions")
plt.legend()
plt.grid()
plt.show()




In [15]:
import torch
saved_outputs = torch.load("saved_outputs_0_200_1000.pth")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Distance function
def distance(x0, x1, lp=2):
    return ((x0 - x1).abs()**lp).flatten(1).sum(dim=-1)
label_df = pd.read_csv("tabddpm_white_box/train/tabddpm_1/challenge_label.csv")

# Assume saved_outputs is your dictionary with noise tensors
# Assume label_df is a DataFrame with one column containing 1 (member) or 0 (non-member)
labels = label_df.iloc[:, 0].values  # Convert to numpy array

# Separate members and non-members
members = labels == 1
non_members = labels == 0

# Plot noise magnitude per point for different T values
plt.figure(figsize=(10, 6))
for T, noise in saved_outputs.items():
    noise_magnitude = noise.norm(dim=1)  # Compute magnitude of noise per point
    plt.plot([T] * len(noise_magnitude[members]), noise_magnitude[members], 'bo', alpha=0.02, label='Members' if T == 0 else "")
    plt.plot([T] * len(noise_magnitude[non_members]), noise_magnitude[non_members], 'ro', alpha=0.02, label='Non-Members' if T == 0 else "")

plt.xlabel("T values")
plt.ylabel("Noise Magnitude")
plt.title("Noise Magnitude per Point Over Different T values")
plt.legend()
plt.show()

# Compute distances for T = 0 vs 1000 and T = 0 vs 200
distances_0_1999 = distance(saved_outputs[0], saved_outputs[1999])
distances_0_1800= distance(saved_outputs[0], saved_outputs[1800])
distances_0_1600 = distance(saved_outputs[0], saved_outputs[1600])
distances_0_1400 = distance(saved_outputs[0], saved_outputs[1400])
distances_0_1200 = distance(saved_outputs[0], saved_outputs[1200])
distances_0_1000 = distance(saved_outputs[0], saved_outputs[1000])
distances_0_800 = distance(saved_outputs[0], saved_outputs[800])
distances_0_600 = distance(saved_outputs[0], saved_outputs[600])
distances_0_400 = distance(saved_outputs[0], saved_outputs[400])
distances_0_200 = distance(saved_outputs[0], saved_outputs[200])

# Separate distances for members and non-members
dist_0_1000_members = distances_0_1000[members]
dist_0_1000_non_members = distances_0_1000[non_members]

dist_0_200_members = distances_0_400[members]
dist_0_200_non_members = distances_0_400[non_members]

# Plot histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].hist(dist_0_1000_members, bins=50, alpha=0.6, color='b', label="Members")
axes[0].hist(dist_0_1000_non_members, bins=50, alpha=0.6, color='r', label="Non-Members")
axes[0].set_title("Distance between T=0 and T=1999")
axes[0].set_xlabel("Distance")
axes[0].set_ylabel("Frequency")
axes[0].legend()

axes[1].hist(dist_0_200_members, bins=50, alpha=0.6, color='b', label="Members")
axes[1].hist(dist_0_200_non_members, bins=50, alpha=0.6, color='r', label="Non-Members")
axes[1].set_title("Distance between T=0 and T=400")
axes[1].set_xlabel("Distance")
axes[1].set_ylabel("Frequency")
axes[1].legend()

plt.tight_layout()
plt.show()


In [ ]:

# Normalize distances to [0,1] range for probability scores
def normalize_scores(member_distances, nonmember_distances):
    max_dist = max(member_distances.max().item(), nonmember_distances.max().item())
    member_probs = 1 - (member_distances / max_dist)
    nonmember_probs = 1 - (nonmember_distances / max_dist)
    return member_probs, nonmember_probs

# Compute scores
member_scores, nonmember_scores = normalize_scores(distances_0_400[members], distances_0_400[non_members])

# Create labels (1 for members, 0 for non-members)
labels = torch.cat([torch.ones_like(member_scores), torch.zeros_like(nonmember_scores)]).long()
scores = torch.cat([member_scores, nonmember_scores])

# Compute predictions
predictions = (scores).long()

# Save predictions and scores to CSV
df = pd.DataFrame({
    'score': scores.numpy(),
})
df.to_csv(os.path.join(path, "predictions.csv"), index=False, header=False)

# Compute AUROC
auroc = BinaryAUROC()(scores, labels).item()

# Compute ROC curve
roc = BinaryROC()(scores, labels)
fpr, tpr, _ = roc  # Unpack outputs

# Extract TPR at 10% FPR
fpr_threshold = 0.1
tpr_at_1_fpr = tpr[(fpr < fpr_threshold).sum() - 1].item()

# Print results
print(f"AUROC: {auroc:.4f}")
print(f"TPR at 10% FPR: {tpr_at_1_fpr:.4f}")
print("Predictions saved to predictions.csv")



## Package Imports and Evironment Setup

Ensure that you have installed the proper dependenices to run the notebook. The environment installation instructions are available [here](https://github.com/VectorInstitute/MIDSTModels/tree/main/starter_kits). Now that we have verfied we have the proper packages installed, lets import them and define global variables:

In [ ]:
%cd starter_kits/

In [49]:

TABDDPM_DATA_DIR = "tabddpm_white_box"
TABSYN_DATA_DIR = "tabsyn_white_box"

In [10]:
def attack(base_dir, attacker_name="PIA", attack_num=30, interval=10, lp=3):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    # logger.addHandler(RichHandler())

    logger.info("loading the attacked model...")

    # Initialize attacker
    phases = ["train"]
    # phases = ["dev", "final"]
    
    logger.info("attack start...")
    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
            if "30" not in str(path):
                continue

            model = tabddpm_whitebox_load_pretrained(path)
            attacker = ATTACKERS[attacker_name](
                torch.from_numpy(np.linspace(FLAGS.beta_1, FLAGS.beta_T, FLAGS.T)).to(DEVICE), interval, attack_num, EpsGetter(model), lp=lp)

            challenge_points = get_challenge_points(path)
            challenge_labels = get_challenge_labels(path)
            raw_predictions = torch.stack([attacker(cp.to(DEVICE).float()) for cp in challenge_points])
            raw_predictions_means = raw_predictions.mean(dim=1).cpu().detach().numpy()
            challenge_labels_np = challenge_labels.values.squeeze()

            non_member_distances = raw_predictions_means[challenge_labels_np == 0]
            member_distances = raw_predictions_means[challenge_labels_np == 1]

            # Create figure with three subplots
            fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), height_ratios=[1, 1, 1.5])
            
            # Plot 1: Full distribution
            bins = np.linspace(min(raw_predictions_means), max(raw_predictions_means), 30)
            ax1.hist(non_member_distances, bins=bins, alpha=0.6, 
                    label=f'Non-member (n={len(non_member_distances)})', 
                    color='blue')
            ax1.hist(member_distances, bins=bins, alpha=0.6,
                    label=f'Member (n={len(member_distances)})', 
                    color='red')
            
            ax1.set_xlabel('Distance')
            ax1.set_ylabel('Count')
            ax1.set_title('Full Distribution of Distances')
            ax1.legend()
            ax1.grid(True, alpha=0.3)

            # First zoom: Find the densest region
            all_data = np.concatenate([member_distances, non_member_distances])
            Q1 = np.percentile(all_data, 25)
            Q3 = np.percentile(all_data, 75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 0.5 * IQR
            upper_bound = Q3 + 0.5 * IQR

            # Filter data for first zoom
            non_member_filtered = non_member_distances[
                (non_member_distances >= lower_bound) & 
                (non_member_distances <= upper_bound)
            ]
            member_filtered = member_distances[
                (member_distances >= lower_bound) & 
                (member_distances <= upper_bound)
            ]

            # Plot 2: First zoom level
            detailed_bins = np.linspace(lower_bound, upper_bound, 50)
            ax2.hist(non_member_filtered, bins=detailed_bins, alpha=0.6,
                    label=f'Non-member (n={len(non_member_filtered)})', 
                    color='blue')
            ax2.hist(member_filtered, bins=detailed_bins, alpha=0.6,
                    label=f'Member (n={len(member_filtered)})', 
                    color='red')
            
            ax2.set_xlabel('Distance')
            ax2.set_ylabel('Count')
            ax2.set_title('First Zoom Level\n'
                        f'(Range: {lower_bound:.2f} to {upper_bound:.2f})')
            ax2.legend()
            ax2.grid(True, alpha=0.3)

            # Second zoom: Find the even denser region
            filtered_data = np.concatenate([member_filtered, non_member_filtered])
            Q1_filtered = np.percentile(filtered_data, 25)
            Q3_filtered = np.percentile(filtered_data, 75)
            IQR_filtered = Q3_filtered - Q1_filtered
            lower_bound_filtered = Q1_filtered - 0.25 * IQR_filtered  # Using tighter bounds
            upper_bound_filtered = Q3_filtered + 0.25 * IQR_filtered

            # Filter data for second zoom
            non_member_filtered_2 = non_member_filtered[
                (non_member_filtered >= lower_bound_filtered) & 
                (non_member_filtered <= upper_bound_filtered)
            ]
            member_filtered_2 = member_filtered[
                (member_filtered >= lower_bound_filtered) & 
                (member_filtered <= upper_bound_filtered)
            ]

            # Plot 3: Second zoom level with very fine bins
            very_detailed_bins = np.linspace(lower_bound_filtered, upper_bound_filtered, 100)
            ax3.hist(non_member_filtered_2, bins=very_detailed_bins, alpha=0.6,
                    label=f'Non-member (n={len(non_member_filtered_2)})', 
                    color='blue')
            ax3.hist(member_filtered_2, bins=very_detailed_bins, alpha=0.6,
                    label=f'Member (n={len(member_filtered_2)})', 
                    color='red')
            
            ax3.set_xlabel('Distance')
            ax3.set_ylabel('Count')
            ax3.set_title('Second Zoom Level (Finest Detail)\n'
                        f'(Range: {lower_bound_filtered:.2f} to {upper_bound_filtered:.2f})')
            ax3.legend()
            ax3.grid(True, alpha=0.3)

            # Add statistical information for the finest zoom level
            stats_text = (
                f'Dense Region Stats:\n'
                f'Non-member:\n'
                f'  Mean: {np.mean(non_member_filtered_2):.3f}\n'
                f'  Std: {np.std(non_member_filtered_2):.3f}\n'
                f'Member:\n'
                f'  Mean: {np.mean(member_filtered_2):.3f}\n'
                f'  Std: {np.std(member_filtered_2):.3f}'
            )
            ax3.text(0.02, 0.98, stats_text, 
                    transform=ax3.transAxes,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            plt.tight_layout()
            plt.savefig(os.path.join(path, f"distance_distribution_T={interval}_lp={lp}.png"), dpi=300, bbox_inches='tight')
            plt.close()

            # Save detailed statistics for all zoom levels
            stats_summary = pd.DataFrame({
                'Metric': ['Full Mean', 'Full Std', 
                        'First Zoom Mean', 'First Zoom Std',
                        'Second Zoom Mean', 'Second Zoom Std',
                        'Points in Densest Region', 'Total Points'],
                'Member': [
                    member_distances.mean(),
                    member_distances.std(),
                    member_filtered.mean(),
                    member_filtered.std(),
                    member_filtered_2.mean(),
                    member_filtered_2.std(),
                    len(member_filtered_2),
                    len(member_distances)
                ],
                'Non-member': [
                    non_member_distances.mean(),
                    non_member_distances.std(),
                    non_member_filtered.mean(),
                    non_member_filtered.std(),
                    non_member_filtered_2.mean(),
                    non_member_filtered_2.std(),
                    len(non_member_filtered_2),
                    len(non_member_distances)
                ]
            })
            
            stats_summary.to_csv(os.path.join(path, f"detailed_statistics_T={interval}_lp={lp}.csv"), index=False)

            # Continue with the original prediction code
            normalized_preds = []
            for pred_batch in raw_predictions:
                binary_preds = (pred_batch <= 1000000).float()
                normalized_preds.append(binary_preds)

            final_predictions = torch.stack(normalized_preds)
            predictions_cpu = final_predictions.cpu().detach().numpy()

            with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                writer = csv.writer(file)
                for value in predictions_cpu.squeeze():
                    writer.writerow([value])



In [ ]:
attack(base_dir="tabddpm_white_box",
        attacker_name="PIAN",
        attack_num=3,
        interval=200,
        lp=4)

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth.
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack.

In [9]:
import numpy as np
from sklearn.metrics import roc_auc_score
import os
from typing import List, Tuple

def safe_load_data(filepath: str, is_prediction: bool = True) -> np.ndarray:
    """
    Safely load data from CSV files with error handling and debugging.
    
    Args:
        filepath: Path to the CSV file
        is_prediction: Whether this is a prediction file (True) or label file (False)
    
    Returns:
        numpy.ndarray: Loaded and validated data
    """
    try:
        if is_prediction:
            # Read the file line by line and parse each array
            predictions = []
            with open(filepath, 'r') as f:
                for line in f:
                    # Remove brackets and split by spaces
                    clean_line = line.strip().replace('[', '').replace(']', '')
                    # Convert space-separated strings to floats
                    row = np.array([float(x) for x in clean_line.split()])
                    predictions.append(row)
            return np.array(predictions)
        else:
            # For label files, skip header and use numpy
            return np.loadtxt(filepath, skiprows=1)
            
    except Exception as e:
        print(f"Error loading {filepath}: {str(e)}")
        print(f"File contents (first few lines):")
        with open(filepath, 'r') as f:
            print(f.read(500))
        raise

def get_tpr_at_fpr(y_true: np.ndarray, y_score: np.ndarray, target_fpr: float = 0.1) -> float:
    """Calculate TPR at a specific FPR threshold."""
    # Input validation
    if not isinstance(y_true, np.ndarray) or not isinstance(y_score, np.ndarray):
        raise TypeError("Inputs must be numpy arrays")
    if y_true.shape != y_score.shape:
        raise ValueError(f"Shape mismatch: y_true {y_true.shape} != y_score {y_score.shape}")
    
    # Sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]
    
    n_neg = np.sum(y_true == 0)
    n_pos = np.sum(y_true == 1)
    
    # Handle edge cases
    if n_neg == 0 or n_pos == 0:
        print("Warning: Found no positive or negative samples")
        return 0.0
        
    tpr = np.cumsum(y_true) / n_pos
    fpr = np.cumsum(1 - y_true) / n_neg
    
    for i in range(len(fpr)):
        if fpr[i] >= target_fpr:
            return tpr[i]
    return 1.0

def evaluate_membership_inference(base_dirs: List[str]) -> Tuple[float, float]:
    """
    Evaluate membership inference attack results across multiple directories.
    
    Args:
        base_dirs: List of base directories containing prediction files
        
    Returns:
        tuple: (best_tpr_at_fpr, best_auc)
    """
    tpr_at_fpr_list = []
    auc_list = []
    
    for base_dir in base_dirs:
        predictions = []
        solutions = []
        root = os.path.join(base_dir, "train")
        
        if not os.path.exists(root):
            print(f"Warning: Directory not found: {root}")
            continue
            
        model_folders = sorted(os.listdir(root), key=lambda d: int(d.split('_')[1]))
        if not model_folders:
            print(f"Warning: No model folders found in {root}")
            continue
        
        # Load and process all predictions and solutions
        for model_folder in model_folders:
            path = os.path.join(root, model_folder)
            pred_path = os.path.join(path, "prediction.csv")
            label_path = os.path.join(path, "challenge_label.csv")
            
            if not (os.path.exists(pred_path) and os.path.exists(label_path)):
                print(f"Warning: Missing files in {path}")
                continue
            
            try:
                # Load predictions
                pred = safe_load_data(pred_path, is_prediction=True)
                predictions.append(pred)
                
                # Load ground truth
                solution = safe_load_data(label_path, is_prediction=False)
                if solution.ndim == 0:
                    solution = solution.reshape(1)
                solutions.append(solution)
                
            except Exception as e:
                print(f"Error processing folder {model_folder}: {str(e)}")
                continue
        
        if not predictions or not solutions:
            print(f"Warning: No valid data found in {base_dir}")
            continue
            
        try:
            # Concatenate all predictions and solutions
            predictions = np.concatenate(predictions)
            solutions = np.concatenate(solutions)
            
            print(f"\nData shapes for {os.path.basename(base_dir)}:")
            print(f"Predictions shape: {predictions.shape}")
            print(f"Solutions shape: {solutions.shape}")
            
            # Calculate metrics for each attacker's predictions
            num_attackers = predictions.shape[1]
            for attacker_idx in range(num_attackers):
                attacker_preds = predictions[:, attacker_idx]
                
                # Basic data validation
                if np.any(np.isnan(attacker_preds)):
                    print(f"Warning: NaN values found in predictions for attacker {attacker_idx}")
                    continue
                    
                # Calculate metrics
                tpr_at_fpr = get_tpr_at_fpr(solutions, attacker_preds)
                tpr_at_fpr_list.append(tpr_at_fpr)
                
                try:
                    auc = roc_auc_score(solutions, attacker_preds)
                    auc_list.append(auc)
                except ValueError as e:
                    print(f"Warning: Could not calculate AUC for attacker {attacker_idx}: {e}")
                    continue
                
                print(f"{os.path.basename(base_dir)} Attacker {attacker_idx + 1}:")
                print(f"  TPR at FPR==10%: {tpr_at_fpr:.4f}")
                print(f"  AUC: {auc:.4f}")
                
        except Exception as e:
            print(f"Error processing directory {base_dir}: {str(e)}")
            continue
    
    # Get best scores
    final_tpr_at_fpr = max(tpr_at_fpr_list) if tpr_at_fpr_list else 0.0
    final_auc = max(auc_list) if auc_list else 0.0
    
    print(f"\nBest scores across all attackers:")
    print(f"Final Train Attack TPR at FPR==10%: {final_tpr_at_fpr:.4f}")
    print(f"Final Train Attack AUC: {final_auc:.4f}")
    
    return final_tpr_at_fpr, final_auc

In [ ]:
base_dirs = [TABDDPM_DATA_DIR]
final_tpr, final_auc = evaluate_membership_inference(base_dirs)

In [ ]:
%cd MIDSTModels/

In [47]:
import csv
import os
import random
import zipfile

import numpy as np
import torch


In [51]:

def parse_numpy_array(array_str):
    """Fix NumPy-style array formatting and convert it to a Python list."""
    array_str = array_str.strip()  # Remove leading/trailing spaces
    array_str = re.sub(r'\s+', ',', array_str)  # Replace spaces with commas
    array_str = array_str.replace("[,", "[").replace(",]", "]")  # Fix misplaced commas
    return eval(array_str)  # Convert string to list

with zipfile.ZipFile("white_box_single_table_submission.zip", 'w') as zipf:
    for phase in ["dev", "final"]:
        for base_dir in [TABDDPM_DATA_DIR]:
            root = os.path.join(base_dir, phase)
            for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
                path = os.path.join(root, model_folder)
                if not os.path.isdir(path): 
                    continue

                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    # Load CSV
                    df = pd.read_csv(file, header=None)

                    # Convert NumPy-style arrays to proper lists
                    df = df[0].apply(lambda x: parse_numpy_array(x) if isinstance(x, str) else x)

                    # Compute mean for each row
                    df_mean = df.apply(lambda x: np.mean(x) if isinstance(x, list) else x)
                    file = os.path.join(path, "prediction.csv")
                    # # Save the new CSV
                    # df_mean.to_csv(file, index=False, header=False)

                    # Add to ZIP
                    arcname = os.path.relpath(file, os.path.dirname(base_dir))
                    zipf.write(file, arcname=arcname)
                else:
                    raise FileNotFoundError(f"`predictions.csv` not found in {path}.")


The generated white_box_single_table_submission.zip can be directly submitted to the dev phase in the CodaBench UI. Although this submission contains your predictions for both the dev and final set, you will only receive feedback on your predictions for the dev phase. The predictions for the final phase will be evaluated once the competiton ends using the most recent submission to the dev phase.